In [22]:
pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ------------------------ --------------- 153.6/250.9 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Vozon Comsof Pvt Ltd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
# Paths and constants
CHROMEDRIVER_PATH = 'chromedriver-win64/chromedriver.exe'
URL = 'https://www.maharashtramedicalcouncil.in/frmRmpList.aspx'
DROPDOWN_VALUE = "1"

# Set up the Chrome WebDriver
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

# Open the webpage
driver.get(URL)

# Wait until the dropdown element is present
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
)

# Select dropdown value
dropdown = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
dropdown.select_by_value(DROPDOWN_VALUE)

# Enter the numbers into the search field
input_search_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtSearch')
numbers_to_insert = "77790"
input_search_field.send_keys(numbers_to_insert)

# Extract the text from the span element
span_element = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lblDisChar')
extracted_text = span_element.text

# Locate the second input field and send the extracted text
input_disp_char_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtDispChar')
input_disp_char_field.send_keys(extracted_text)

search_button = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnSearch')
search_button.click()

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'ctl00_ContentPlaceHolder1_gvList'))
)

# Extract the table rows
rows = driver.find_elements(By.XPATH, "//table[@id='ctl00_ContentPlaceHolder1_gvList']//tr")

# Prepare a list to store the extracted data
data = []

# Loop through the rows, excluding the header row
for row in rows[1:]:
    # Extract the Name, Qualification, Registration No., Registration Date, and Valid Upto Date
    name = row.find_element(By.XPATH, ".//span[contains(@id, 'txtName')]").text
    qualification = row.find_element(By.XPATH, ".//span[contains(@id, 'txtQualification')]").text
    registration_no = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregNo')]").text
    registration_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregdt')]").text
    valid_upto_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtValidupto')]").text
    
    # Append the extracted data to the list
    data.append({
        'Name': name,
        'Qualification': qualification,
        'Registration No.': registration_no,
        'Registration Date': registration_date,
        'Valid Upto Date': valid_upto_date
    })

# Convert the list into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel('medical_registration_data.xlsx', index=False)

# Optionally, close the browser after the task is done
driver.quit()

In [26]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import json

# Paths and constants
CHROMEDRIVER_PATH = 'chromedriver-win64/chromedriver.exe'
URL = 'https://www.maharashtramedicalcouncil.in/frmRmpList.aspx'
DROPDOWN_VALUE = "1"

# List of registration numbers to search for
registration_numbers = ["2017052017", "2017052018", "2017052011", "2017052016", "2017052014"]

# Set up the Chrome WebDriver
service = webdriver.chrome.service.Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)
driver.get(URL)

# Wait until the dropdown element is present
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch')))

# Select the dropdown value
dropdown = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
dropdown.select_by_value(DROPDOWN_VALUE)

# Initialize list to store results
data = []

# Loop through each registration number
for reg_no in registration_numbers:
    # Clear the search input field and enter a new number
    input_search_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtSearch')
    input_search_field.clear()
    input_search_field.send_keys(reg_no)

    # Extract the text from the span element and input it into the second field
    span_element = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lblDisChar')
    extracted_text = span_element.text

    input_disp_char_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtDispChar')
    input_disp_char_field.clear()
    input_disp_char_field.send_keys(extracted_text)

    # Click the search button
    search_button = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnSearch')
    search_button.click()

    # Wait for the search results table to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ctl00_ContentPlaceHolder1_gvList')))

    # Extract the table row (we assume there's only one matching row per search)
    try:
        row = driver.find_element(By.XPATH, "//table[@id='ctl00_ContentPlaceHolder1_gvList']//tr[2]")

        # Extract the relevant data
        name = row.find_element(By.XPATH, ".//span[contains(@id, 'txtName')]").text
        qualification = row.find_element(By.XPATH, ".//span[contains(@id, 'txtQualification')]").text
        registration_no = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregNo')]").text
        registration_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregdt')]").text
        valid_upto_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtValidupto')]").text

        # Append the extracted data to the list
        data.append({
            'Name': name,
            'Qualification': qualification,
            'Registration No.': registration_no,
            'Registration Date': registration_date,
            'Valid Upto Date': valid_upto_date
        })

    except Exception as e:
        print(f"No results found for registration number: {reg_no}")

    # Add a delay to prevent being blocked by the website
    time.sleep(2)

# Close the browser
driver.quit()

# Convert the list of data to a DataFrame
df = pd.DataFrame(data)

# Save the data to CSV
df.to_csv('medical_registration_data.csv', index=False)

# Save the data to JSON
with open('medical_registration_data.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("Data has been saved to 'medical_registration_data.csv' and 'medical_registration_data.json'.")


No results found for registration number: 2017052018
No results found for registration number: 2017052011
No results found for registration number: 2017052016
Data has been saved to 'medical_registration_data.csv' and 'medical_registration_data.json'.


In [43]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
import time

# Define constants
CHROMEDRIVER_PATH = 'chromedriver-win64/chromedriver.exe'
URL = 'https://www.maharashtramedicalcouncil.in/frmRmpList.aspx'
DROPDOWN_VALUE = "1"
CSV_FILE_PATH = 'regestration_no.csv'  # Path to your CSV file

# Set up the Chrome WebDriver
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

# Open the webpage
driver.get(URL)

# Wait until the dropdown element is present
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
)

# Select dropdown value
dropdown = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
dropdown.select_by_value(DROPDOWN_VALUE)

# Read registration numbers from CSV
df_csv = pd.read_csv(CSV_FILE_PATH)
registration_numbers = df_csv['Registration No'].tolist()

# Prepare a list to store the extracted data
data = []

for number in registration_numbers:
    try:
        # Enter the numbers into the search field
        input_search_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtSearch')
        input_search_field.clear()  # Clear any previous input
        input_search_field.send_keys(number)
        
        # Extract the text from the span element
        span_element = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lblDisChar')
        extracted_text = span_element.text

        # Locate the second input field and send the extracted text
        input_disp_char_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtDispChar')
        input_disp_char_field.clear()  # Clear any previous input
        input_disp_char_field.send_keys(extracted_text)

        # Click the search button
        search_button = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnSearch')
        search_button.click()

        # Wait until the table is present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'ctl00_ContentPlaceHolder1_gvList'))
        )

        # Extract the table rows
        rows = driver.find_elements(By.XPATH, "//table[@id='ctl00_ContentPlaceHolder1_gvList']//tr")

        # Extract data for the first row (excluding header)
        if len(rows) > 1:
            row = rows[1]
            name = row.find_element(By.XPATH, ".//span[contains(@id, 'txtName')]").text
            qualification = row.find_element(By.XPATH, ".//span[contains(@id, 'txtQualification')]").text
            registration_no = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregNo')]").text
            registration_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregdt')]").text
            valid_upto_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtValidupto')]").text
            
            # Prepare the data in the required format
            result = {
                'Registration Number': number,  # Changed key name
                'Name': name,
                'Qualification': qualification,
                'Registration No.': registration_no,  # Retain this key for clarity
                'Registration Date': registration_date,
                'Valid Upto Date': valid_upto_date
            }
            
            # Print the result
            print(result)
            
            # Append the extracted data to the list
            data.append(result)
        else:
            # If no data is found, prepare and print a record with no data
            result = {
                'Registration Number': number,  # Changed key name
                'Name': 'N/A',
                'Qualification': 'N/A',
                'Registration No.': 'N/A',
                'Registration Date': 'N/A',
                'Valid Upto Date': 'N/A'
            }
            
            print(result)
            
            # Append the record with no data
            data.append(result)

    except StaleElementReferenceException as e:
        print(f"StaleElementReferenceException for Registration No. {number}: {e}")
        time.sleep(2)  # Delay before retrying
        continue

# Convert the list into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel('medical_registration_data.xlsx', index=False)

# Optionally, close the browser after the task is done
driver.quit()


{'Registration Number': 2017052017, 'Name': 'ARJUN PRATAP JOSHI', 'Qualification': 'M.B.B.S.,', 'Registration No.': '2017052017', 'Registration Date': '15/05/2017', 'Valid Upto Date': '15/05/2022'}
StaleElementReferenceException for Registration No. 2017052018: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF720FD9642+30946]
	(No symbol) [0x00007FF720F8E3D9]
	(No symbol) [0x00007FF720E86FDA]
	(No symbol) [0x00007FF720E8DA57]
	(No symbol) [0x00007FF720E8FD31]
	(No symbol) [0x00007FF720E8FDD0]
	(No symbol) [0x00007FF720ED226B]
	(No symbol) [0x00007FF720EFCA6A]
	(No symbol) [0x00007FF720ECBBB6]
	(No symbol) [0x00007FF720EFCC80]
	(No symbol) [0x00007FF720F1B041]
	(No symbol) [0x00007FF720EFC813]
	(No symbol) [0x00007FF720ECA6E5]

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import json

# Paths and constants
CHROMEDRIVER_PATH = 'chromedriver-win64/chromedriver.exe'
URL = 'https://www.maharashtramedicalcouncil.in/frmRmpList.aspx'
DROPDOWN_VALUE = "1"

def fetch_registration_data(numbers_to_insert):
    # Set up the Chrome WebDriver
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    # Open the webpage
    driver.get(URL)

    # Wait until the dropdown element is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
    )

    # Select dropdown value
    dropdown = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
    dropdown.select_by_value(DROPDOWN_VALUE)

    # Enter the numbers into the search field
    input_search_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtSearch')
    input_search_field.send_keys(numbers_to_insert)

    # Extract the text from the span element
    span_element = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lblDisChar')
    extracted_text = span_element.text

    # Locate the second input field and send the extracted text
    input_disp_char_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtDispChar')
    input_disp_char_field.send_keys(extracted_text)

    # Click the search button
    search_button = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnSearch')
    search_button.click()

    # Wait until the table is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ctl00_ContentPlaceHolder1_gvList'))
    )

    # Extract the table rows
    rows = driver.find_elements(By.XPATH, "//table[@id='ctl00_ContentPlaceHolder1_gvList']//tr")

    # Prepare a list to store the extracted data
    data = []

    # Loop through the rows, excluding the header row
    for row in rows[1:]:
        # Extract the Name, Qualification, Registration No., Registration Date, and Valid Upto Date
        name = row.find_element(By.XPATH, ".//span[contains(@id, 'txtName')]").text
        qualification = row.find_element(By.XPATH, ".//span[contains(@id, 'txtQualification')]").text
        registration_no = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregNo')]").text
        registration_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregdt')]").text
        valid_upto_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtValidupto')]").text

        # Append the extracted data to the list
        data.append({
            'Name': name,
            'Qualification': qualification,
            'Registration No.': registration_no,
            'Registration Date': registration_date,
            'Valid Upto Date': valid_upto_date
        })

    # Close the browser after the task is done
    driver.quit()

    # Convert the list into JSON
    json_output = json.dumps(data, indent=4)
    
    return json_output

# Example usage with a changeable number
numbers_to_insert = "77790"
json_data = fetch_registration_data(numbers_to_insert)

# Print the JSON output
print(json_data)


[
    {
        "Name": "MEHTA SHREYA MANOJ",
        "Qualification": "M.B.B.S., M.D. (Med. & Thera.),",
        "Registration No.": "77790",
        "Registration Date": "05/10/1995",
        "Valid Upto Date": "28/02/2027"
    }
]


: 